In [6]:
import os

os.chdir('/Users/gabriele/PycharmProjects/RecSys')

from src.HyperparameterTuning.SearchAbstractClass import SearchInputRecommenderArgs
from src.Recommenders.GraphBased.RP3betaRecommender import RP3betaRecommender
from Utils.utils import create_URM,create_ICM_with_events,combine_matrices
from src.Evaluation.Evaluator import EvaluatorHoldout
from src.Data_manager.split_functions.split_train_validation_random_holdout import split_train_in_two_percentage_global_sample
from skopt.space import Real, Integer, Categorical
import numpy as np
##Best config is 81: 'topK': 82, 'alpha': 0.39354058844901263, 'beta': 0.5811550127865287, 'implicit': True, 'normalize_similarity': True

In [4]:
URM = create_URM()
ICM = create_ICM_with_events()

URM_train_validation,URM_test = split_train_in_two_percentage_global_sample(URM,train_percentage=0.85)
URM_train, URM_validation = split_train_in_two_percentage_global_sample(URM_train_validation,train_percentage=0.85)

combined_matrices = combine_matrices(ICM=ICM, URM=URM_train)




In [8]:
##SELECT 25% OF USERS WITH LONGEST PROFILES TO FINE-TUNE THE HYPERPARAMETERS
users_to_ignore=[]
profile_length = np.ediff1d(URM_train.indptr)
block_size = int(len(profile_length) * 0.25)

start_pos = 3 * block_size
end_pos = min(4 * block_size, len(profile_length))
sorted_users = np.argsort(profile_length)

users_in_group = sorted_users[start_pos:end_pos]

users_in_group_p_len = profile_length[users_in_group]
sorted_users = np.argsort(profile_length)

users_not_in_group_flag = np.isin(sorted_users, users_in_group, invert=True)

In [9]:
evaluator_test = EvaluatorHoldout(URM_test, cutoff_list=[10])
evaluator_validation = EvaluatorHoldout(URM_validation, cutoff_list=[10],ignore_users=sorted_users[users_not_in_group_flag])

EvaluatorHoldout: Ignoring 13640 ( 0.1%) Users that have less than 1 test interactions
EvaluatorHoldout: Ignoring 13632 ( 0.1%) Users that have less than 1 test interactions
EvaluatorHoldout: Ignoring 10238 Users


In [11]:
from src.HyperparameterTuning.SearchBayesianSkopt import SearchBayesianSkopt
hyperparameters_range_dictionary = {
    "topK": Integer(80,120),
    "alpha": Real(0.3,0.7),
    "beta": Real(0.3,0.7),
    "implicit": Categorical([True]),
    "normalize_similarity": Categorical([True])
}
recommender_input_args = SearchInputRecommenderArgs(
    CONSTRUCTOR_POSITIONAL_ARGS = [combined_matrices],
    CONSTRUCTOR_KEYWORD_ARGS = {},
    FIT_POSITIONAL_ARGS = [],
    FIT_KEYWORD_ARGS = {}
)
recommender_input_args_last_test = SearchInputRecommenderArgs(
    CONSTRUCTOR_POSITIONAL_ARGS = [URM_train_validation],
    CONSTRUCTOR_KEYWORD_ARGS = {},
    FIT_POSITIONAL_ARGS = [],
    FIT_KEYWORD_ARGS = {}
)

tuning_class = SearchBayesianSkopt(recommender_class=RP3betaRecommender,
                                   evaluator_validation=evaluator_validation,
                                   evaluator_test=evaluator_test)

n_cases = 100
n_random_starts = n_cases*0.3
output_folder_path = "logs/"

tuning_class.search(recommender_input_args=recommender_input_args,
                    recommender_input_args_last_test=recommender_input_args_last_test,
                    hyperparameter_search_space=hyperparameters_range_dictionary,
                    metric_to_optimize="MAP",
                    cutoff_to_optimize= 10,
                    n_cases= n_cases,
                    n_random_starts=n_random_starts,
                    output_folder_path=output_folder_path,
                    output_file_name_root= RP3betaRecommender.RECOMMENDER_NAME,
                    save_model="best"
                    )

Iteration No: 1 started. Evaluating function at random point.
SearchBayesianSkopt: Testing config: {'topK': 90, 'alpha': 0.6784172521669138, 'beta': 0.4639390300044524, 'implicit': True, 'normalize_similarity': True}
RP3betaRecommender: URM Detected 10245 ( 2.8%) users with no interactions.
EvaluatorHoldout: Processed 3412 (100.0%) in 7.77 sec. Users per second: 439
SearchBayesianSkopt: New best config found. Config 0: {'topK': 90, 'alpha': 0.6784172521669138, 'beta': 0.4639390300044524, 'implicit': True, 'normalize_similarity': True} - results: PRECISION: 0.2349941, PRECISION_RECALL_MIN_DEN: 0.2349941, RECALL: 0.0267839, MAP: 0.1110268, MAP_MIN_DEN: 0.1110268, MRR: 0.4505043, NDCG: 0.2381499, F1: 0.0480869, HIT_RATE: 0.9273154, ARHR_ALL_HITS: 0.7057652, NOVELTY: 0.0058598, AVERAGE_POPULARITY: 0.4277142, DIVERSITY_MEAN_INTER_LIST: 0.8786688, DIVERSITY_HERFINDAHL: 0.9878411, COVERAGE_ITEM: 0.0830611, COVERAGE_ITEM_CORRECT: 0.0482862, COVERAGE_USER: 1.0000000, COVERAGE_USER_CORRECT: 0.92

RP3betaRecommender: Saving complete
Iteration No: 4 ended. Evaluation done at random point.
Time taken: 44.5967
Function value obtained: -0.1119
Current minimum: -0.1119
Iteration No: 5 started. Evaluating function at random point.
SearchBayesianSkopt: Testing config: {'topK': 105, 'alpha': 0.5661725816153947, 'beta': 0.3791734289073547, 'implicit': True, 'normalize_similarity': True}
RP3betaRecommender: URM Detected 10245 ( 2.8%) users with no interactions.
EvaluatorHoldout: Processed 3412 (100.0%) in 11.19 sec. Users per second: 305
SearchBayesianSkopt: Config 4 is suboptimal. Config: {'topK': 105, 'alpha': 0.5661725816153947, 'beta': 0.3791734289073547, 'implicit': True, 'normalize_similarity': True} - results: PRECISION: 0.2346717, PRECISION_RECALL_MIN_DEN: 0.2346717, RECALL: 0.0267479, MAP: 0.1106914, MAP_MIN_DEN: 0.1106914, MRR: 0.4504692, NDCG: 0.2378578, F1: 0.0480222, HIT_RATE: 0.9296600, ARHR_ALL_HITS: 0.7048566, NOVELTY: 0.0059054, AVERAGE_POPULARITY: 0.4065745, DIVERSITY_ME

EvaluatorHoldout: Processed 3412 (100.0%) in 8.74 sec. Users per second: 391
SearchBayesianSkopt: Config 9 is suboptimal. Config: {'topK': 116, 'alpha': 0.4618705260865352, 'beta': 0.3953436396535325, 'implicit': True, 'normalize_similarity': True} - results: PRECISION: 0.2360199, PRECISION_RECALL_MIN_DEN: 0.2360199, RECALL: 0.0269020, MAP: 0.1108211, MAP_MIN_DEN: 0.1108211, MRR: 0.4505038, NDCG: 0.2385467, F1: 0.0482987, HIT_RATE: 0.9296600, ARHR_ALL_HITS: 0.7052781, NOVELTY: 0.0058422, AVERAGE_POPULARITY: 0.4295604, DIVERSITY_MEAN_INTER_LIST: 0.8800300, DIVERSITY_HERFINDAHL: 0.9879772, COVERAGE_ITEM: 0.0742012, COVERAGE_ITEM_CORRECT: 0.0454067, COVERAGE_USER: 1.0000000, COVERAGE_USER_CORRECT: 0.9296600, DIVERSITY_GINI: 0.0136901, SHANNON_ENTROPY: 7.8731020, RATIO_DIVERSITY_HERFINDAHL: 0.9883377, RATIO_DIVERSITY_GINI: 0.0511720, RATIO_SHANNON_ENTROPY: 0.6303547, RATIO_AVERAGE_POPULARITY: 2.7105627, RATIO_NOVELTY: 0.7416547, 

Iteration No: 10 ended. Evaluation done at random point.
Ti

EvaluatorHoldout: Processed 3412 (100.0%) in 7.81 sec. Users per second: 437
SearchBayesianSkopt: Config 15 is suboptimal. Config: {'topK': 102, 'alpha': 0.5132967776845074, 'beta': 0.5016261240403606, 'implicit': True, 'normalize_similarity': True} - results: PRECISION: 0.2358441, PRECISION_RECALL_MIN_DEN: 0.2358441, RECALL: 0.0268945, MAP: 0.1113416, MAP_MIN_DEN: 0.1113416, MRR: 0.4511402, NDCG: 0.2388647, F1: 0.0482831, HIT_RATE: 0.9284877, ARHR_ALL_HITS: 0.7076313, NOVELTY: 0.0057842, AVERAGE_POPULARITY: 0.4543967, DIVERSITY_MEAN_INTER_LIST: 0.8646660, DIVERSITY_HERFINDAHL: 0.9864413, COVERAGE_ITEM: 0.0787973, COVERAGE_ITEM_CORRECT: 0.0460712, COVERAGE_USER: 1.0000000, COVERAGE_USER_CORRECT: 0.9284877, DIVERSITY_GINI: 0.0132471, SHANNON_ENTROPY: 7.7221282, RATIO_DIVERSITY_HERFINDAHL: 0.9868012, RATIO_DIVERSITY_GINI: 0.0495158, RATIO_SHANNON_ENTROPY: 0.6182671, RATIO_AVERAGE_POPULARITY: 2.8672817, RATIO_NOVELTY: 0.7342970, 

Iteration No: 16 ended. Evaluation done at random point.
T

EvaluatorHoldout: Processed 3412 (100.0%) in 7.65 sec. Users per second: 446
SearchBayesianSkopt: Config 21 is suboptimal. Config: {'topK': 114, 'alpha': 0.48296338683599155, 'beta': 0.3259618332734185, 'implicit': True, 'normalize_similarity': True} - results: PRECISION: 0.2333822, PRECISION_RECALL_MIN_DEN: 0.2333822, RECALL: 0.0266166, MAP: 0.1115976, MAP_MIN_DEN: 0.1115976, MRR: 0.4586525, NDCG: 0.2387357, F1: 0.0477835, HIT_RATE: 0.9296600, ARHR_ALL_HITS: 0.7135229, NOVELTY: 0.0059708, AVERAGE_POPULARITY: 0.3799795, DIVERSITY_MEAN_INTER_LIST: 0.9170199, DIVERSITY_HERFINDAHL: 0.9916751, COVERAGE_ITEM: 0.0779667, COVERAGE_ITEM_CORRECT: 0.0484523, COVERAGE_USER: 1.0000000, COVERAGE_USER_CORRECT: 0.9296600, DIVERSITY_GINI: 0.0164639, SHANNON_ENTROPY: 8.2899925, RATIO_DIVERSITY_HERFINDAHL: 0.9920370, RATIO_DIVERSITY_GINI: 0.0615399, RATIO_SHANNON_ENTROPY: 0.6637328, RATIO_AVERAGE_POPULARITY: 2.3977030, RATIO_NOVELTY: 0.7579795, 

Iteration No: 22 ended. Evaluation done at random point.


...

EvaluatorHoldout: Processed 3412 (100.0%) in 7.67 sec. Users per second: 445
SearchBayesianSkopt: Config 74 is suboptimal. Config: {'topK': 120, 'alpha': 0.3027042136675995, 'beta': 0.37074147805543584, 'implicit': True, 'normalize_similarity': True} - results: PRECISION: 0.2356682, PRECISION_RECALL_MIN_DEN: 0.2356682, RECALL: 0.0268210, MAP: 0.1110733, MAP_MIN_DEN: 0.1110733, MRR: 0.4519590, NDCG: 0.2387205, F1: 0.0481608, HIT_RATE: 0.9331770, ARHR_ALL_HITS: 0.7072058, NOVELTY: 0.0058559, AVERAGE_POPULARITY: 0.4230585, DIVERSITY_MEAN_INTER_LIST: 0.8833031, DIVERSITY_HERFINDAHL: 0.9883044, COVERAGE_ITEM: 0.0733706, COVERAGE_ITEM_CORRECT: 0.0460712, COVERAGE_USER: 1.0000000, COVERAGE_USER_CORRECT: 0.9331770, DIVERSITY_GINI: 0.0138739, SHANNON_ENTROPY: 7.9106073, RATIO_DIVERSITY_HERFINDAHL: 0.9886651, RATIO_DIVERSITY_GINI: 0.0518589, RATIO_SHANNON_ENTROPY: 0.6333576, RATIO_AVERAGE_POPULARITY: 2.6695354, RATIO_NOVELTY: 0.7433929, 

Iteration No: 75 ended. Search finished for the next opti

EvaluatorHoldout: Processed 3412 (100.0%) in 7.56 sec. Users per second: 451
SearchBayesianSkopt: Config 80 is suboptimal. Config: {'topK': 98, 'alpha': 0.4264253123645612, 'beta': 0.34043580932304324, 'implicit': True, 'normalize_similarity': True} - results: PRECISION: 0.2334994, PRECISION_RECALL_MIN_DEN: 0.2334994, RECALL: 0.0266432, MAP: 0.1117241, MAP_MIN_DEN: 0.1117241, MRR: 0.4565316, NDCG: 0.2386125, F1: 0.0478290, HIT_RATE: 0.9290739, ARHR_ALL_HITS: 0.7125159, NOVELTY: 0.0059911, AVERAGE_POPULARITY: 0.3748176, DIVERSITY_MEAN_INTER_LIST: 0.9181981, DIVERSITY_HERFINDAHL: 0.9917929, COVERAGE_ITEM: 0.0832826, COVERAGE_ITEM_CORRECT: 0.0515532, COVERAGE_USER: 1.0000000, COVERAGE_USER_CORRECT: 0.9290739, DIVERSITY_GINI: 0.0180136, SHANNON_ENTROPY: 8.3898847, RATIO_DIVERSITY_HERFINDAHL: 0.9921548, RATIO_DIVERSITY_GINI: 0.0673327, RATIO_SHANNON_ENTROPY: 0.6717306, RATIO_AVERAGE_POPULARITY: 2.3651309, RATIO_NOVELTY: 0.7605602, 

Iteration No: 81 ended. Search finished for the next optim

EvaluatorHoldout: Processed 3412 (100.0%) in 7.91 sec. Users per second: 431
SearchBayesianSkopt: Config 86 is suboptimal. Config: {'topK': 80, 'alpha': 0.5198605397156104, 'beta': 0.5953410886593531, 'implicit': True, 'normalize_similarity': True} - results: PRECISION: 0.2364302, PRECISION_RECALL_MIN_DEN: 0.2364302, RECALL: 0.0269730, MAP: 0.1124312, MAP_MIN_DEN: 0.1124312, MRR: 0.4569175, NDCG: 0.2404274, F1: 0.0484219, HIT_RATE: 0.9270223, ARHR_ALL_HITS: 0.7148605, NOVELTY: 0.0057522, AVERAGE_POPULARITY: 0.4690659, DIVERSITY_MEAN_INTER_LIST: 0.8670427, DIVERSITY_HERFINDAHL: 0.9866789, COVERAGE_ITEM: 0.0862728, COVERAGE_ITEM_CORRECT: 0.0463481, COVERAGE_USER: 1.0000000, COVERAGE_USER_CORRECT: 0.9270223, DIVERSITY_GINI: 0.0134698, SHANNON_ENTROPY: 7.6935837, RATIO_DIVERSITY_HERFINDAHL: 0.9870389, RATIO_DIVERSITY_GINI: 0.0503485, RATIO_SHANNON_ENTROPY: 0.6159817, RATIO_AVERAGE_POPULARITY: 2.9598456, RATIO_NOVELTY: 0.7302314, 

Iteration No: 87 ended. Search finished for the next optima

EvaluatorHoldout: Processed 3412 (100.0%) in 7.88 sec. Users per second: 433
SearchBayesianSkopt: Config 92 is suboptimal. Config: {'topK': 80, 'alpha': 0.4256518920766879, 'beta': 0.5853454366744475, 'implicit': True, 'normalize_similarity': True} - results: PRECISION: 0.2372509, PRECISION_RECALL_MIN_DEN: 0.2372509, RECALL: 0.0270862, MAP: 0.1127100, MAP_MIN_DEN: 0.1127100, MRR: 0.4562494, NDCG: 0.2408139, F1: 0.0486214, HIT_RATE: 0.9290739, ARHR_ALL_HITS: 0.7147384, NOVELTY: 0.0057608, AVERAGE_POPULARITY: 0.4660142, DIVERSITY_MEAN_INTER_LIST: 0.8677699, DIVERSITY_HERFINDAHL: 0.9867516, COVERAGE_ITEM: 0.0863835, COVERAGE_ITEM_CORRECT: 0.0466250, COVERAGE_USER: 1.0000000, COVERAGE_USER_CORRECT: 0.9290739, DIVERSITY_GINI: 0.0137823, SHANNON_ENTROPY: 7.7208573, RATIO_DIVERSITY_HERFINDAHL: 0.9871116, RATIO_DIVERSITY_GINI: 0.0515164, RATIO_SHANNON_ENTROPY: 0.6181654, RATIO_AVERAGE_POPULARITY: 2.9405892, RATIO_NOVELTY: 0.7313257, 

Iteration No: 93 ended. Search finished for the next optima

EvaluatorHoldout: Processed 3412 (100.0%) in 7.93 sec. Users per second: 430
SearchBayesianSkopt: Config 98 is suboptimal. Config: {'topK': 80, 'alpha': 0.46552444678531973, 'beta': 0.6136053438505591, 'implicit': True, 'normalize_similarity': True} - results: PRECISION: 0.2363716, PRECISION_RECALL_MIN_DEN: 0.2363716, RECALL: 0.0269501, MAP: 0.1125884, MAP_MIN_DEN: 0.1125884, MRR: 0.4610147, NDCG: 0.2410101, F1: 0.0483837, HIT_RATE: 0.9279015, ARHR_ALL_HITS: 0.7186758, NOVELTY: 0.0057293, AVERAGE_POPULARITY: 0.4762640, DIVERSITY_MEAN_INTER_LIST: 0.8656959, DIVERSITY_HERFINDAHL: 0.9865442, COVERAGE_ITEM: 0.0856637, COVERAGE_ITEM_CORRECT: 0.0453513, COVERAGE_USER: 1.0000000, COVERAGE_USER_CORRECT: 0.9279015, DIVERSITY_GINI: 0.0129689, SHANNON_ENTROPY: 7.6342743, RATIO_DIVERSITY_HERFINDAHL: 0.9869042, RATIO_DIVERSITY_GINI: 0.0484759, RATIO_SHANNON_ENTROPY: 0.6112332, RATIO_AVERAGE_POPULARITY: 3.0052662, RATIO_NOVELTY: 0.7273291, 

Iteration No: 99 ended. Search finished for the next optim